**Maniya Yash Rajeshbhai (B20CS033)**

# **Cryptography HW2**
## **Question 1**

In [1]:
%pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


### Importing Libraries

In [2]:
import sys
import os
from Crypto.Cipher import AES
import base64

from Crypto import Random


### Padding Message

In [3]:
Block_Size = 16

def pad_message(m):

    paddedMessage = m.encode()
    length = len(m)

    p = 16 - length % 16

    pad = p.to_bytes(1, 'big')

    paddedMessage += (pad*p)

    return paddedMessage


### Encrypting Message

In [4]:
global secret_key, IV, encryptor, decryptor


In [5]:
def initialize_encryptor():
    global secret_key, IV, encryptor
    secret_key = os.urandom(16)
    IV = os.urandom(16)
    encryptor = AES.new(secret_key, mode=AES.MODE_CBC, IV=IV)

def cbc_encrypt(m):
    initialize_encryptor()
    c = encryptor.encrypt(m)
    return c



In [6]:
plaintext = "B20CS033YashRajeshHiralSurat"
print("Length of plaintext without padding :", len(plaintext))


Length of plaintext without padding : 28


In [7]:
ciphertext = cbc_encrypt(pad_message(plaintext))
print("Ciphertext obtained after encryption of plaintext is : ", ciphertext)


Ciphertext obtained after encryption of plaintext is :  b's\x06\xbeg\\\xce\x82\x8f\xdb\xd8\xb0\x10\x07\x17\xbc\xb9C\xd5\x83&D\x9eI\xacz;}1>l\x14\x02'


### Decrypting Message using Key

In [8]:
def cbc_decryption(ciphertext):
    decryptor = AES.new(secret_key, mode=AES.MODE_CBC, IV=IV)
    c = ciphertext
    return decryptor.decrypt(c)


In [9]:
print("Decryption of Padded Plaintext with the use of Key : ",
      cbc_decryption(ciphertext))


Decryption of Padded Plaintext with the use of Key :  b'B20CS033YashRajeshHiralSurat\x04\x04\x04\x04'


### Padding Oracle

In [10]:
def isPadded(m):
    p = m[-1]
    i = len(m)-1
    l = len(m)
    if(p > 16 or p < 1):
        return False
    while(i > l-1-p):
        if(m[i] != p):
            return False
        i -= 1
    return True


In [11]:
def padding_oracle(iv, c):
    decrpytor_iv = AES.new(secret_key, mode=AES.MODE_CBC, IV=iv)
    m = decrpytor_iv.decrypt(c)
    return isPadded(m)


### Padding Attack

In [12]:
def padding_attack_block(IV, ciphertext):
    iv = bytearray(IV)
    c = ciphertext
    c0 = bytearray(IV)

    cipher_length = len(c)

    decrypted_message = bytearray(ciphertext)
    padded_bytes = 0

    if(padding_oracle(bytes(iv), c) == True):

        for i in range(16):
            t = iv[i]
            iv[i] = (iv[i]+1) % 256
            if(padding_oracle(bytes(iv), c) == False):
                padded_bytes = cipher_length - i
                iv[i] = t
                break
            iv[i] = t

    bits_identified = padded_bytes

    for i in range(cipher_length-1-padded_bytes, -1, -1):

        bits_identified = (bits_identified+1) % 17

        for j in range(cipher_length-1, i, -1):
            iv[j] = (iv[j] ^ (bits_identified-1) ^ bits_identified) % 256

        padding_oracle(bytes(iv), c)

        for j in range(0, 256):
            iv[i] = j
            if(padding_oracle(bytes(iv), c) == True):
                break

        decrypted_message[i] = (iv[i] ^ bits_identified ^ c0[i]) % 256

    return bytes(decrypted_message)[:16-padded_bytes]


In [13]:
def oracle_attack(IV, ciphertext):
    i = 0
    plaintext_from_attack = ""
    previous_c = IV
    while(i < len(ciphertext)):
        c = ciphertext[i:min(i+16, len(ciphertext))]
        plaintext_from_attack += padding_attack_block(previous_c, c).decode()
        previous_c = c
        i += 16
    return plaintext_from_attack


### Obtaining Plaintext from Attack and Results

In [14]:
plaintext_from_attack = oracle_attack(IV, ciphertext)
print("Plaintext retrieved from the attack :\t", plaintext_from_attack)
print("Original Plaintext : \t\t\t", plaintext)

if plaintext == plaintext_from_attack:
    print("\nPlaintext retrieved from Attack is same as the original one!!")
else:
    print("Aww Ohh! Plaintexts come out to be different")


Plaintext retrieved from the attack :	 B20CS033YashRajeshHiralSurat
Original Plaintext : 			 B20CS033YashRajeshHiralSurat

Plaintext retrieved from Attack is same as the original one!!
